In [ ]:
import open3d as o3d
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from PIL import Image

In [ ]:
# Constants that probably don't change
DEPTH_SCALING_FACTOR    = 1000.0
DEPTH_CUTOFF            = 1.0
VOXEL_SIZE              =0.005

# From handeye.ipynb
cam_to_gripper_rot = np.array([[-7.77766820e-02, -9.44073658e-01, -3.20430518e-01],
 [ 9.96970509e-01, -7.34019190e-02, -2.57286360e-02],
 [ 7.69512542e-04, -3.21460864e-01,  9.46922553e-01]])

cam_to_gripper_trans = np.array([[ 0.10598264], [-0.02439176], [-0.08322135]])

cam_to_gripper_pose = np.eye(4)
cam_to_gripper_pose[:3, :3] = cam_to_gripper_rot
cam_to_gripper_pose[:3, 3] = cam_to_gripper_trans.squeeze()

In [ ]:
def gather_single_sequence_data(base_dir):
    assert os.path.exists(base_dir), f"Path {base_dir} does not exist"
    intrinsic_mat = np.load(os.path.join(base_dir, 'camera_intrinsics.npz'))

    fx, fy = intrinsic_mat['fx'], intrinsic_mat['fy']
    ppx, ppy = intrinsic_mat['ppx'], intrinsic_mat['ppy']

    depth_image_path = os.path.join(base_dir, 'camera_depth')
    assert os.path.exists(depth_image_path), f"Path {depth_image_path} does not exist"
    depth_image_path_list = sorted([os.path.join(depth_image_path, f) for f in os.listdir(depth_image_path) if f.endswith('.npy')])
    depth_image_list = [np.load(f) for f in depth_image_path_list]

    rgb_image_path = os.path.join(base_dir, 'camera_rgb')
    assert os.path.exists(rgb_image_path), f"Path {rgb_image_path} does not exist"
    rgb_image_path_list = sorted([os.path.join(rgb_image_path, f) for f in os.listdir(rgb_image_path) if f.endswith('.png')])
    rgb_image_list = [np.array(Image.open(f).convert('RGB')) for f in rgb_image_path_list]

    assert len(depth_image_list) == len(rgb_image_list)
    assert depth_image_list[0].shape == rgb_image_list[0].shape[:2]

    pose_path = os.path.join(base_dir, 'poses')
    assert os.path.exists(pose_path), f"Path {pose_path} does not exist"
    pose_path_list = sorted([os.path.join(pose_path, f) for f in os.listdir(pose_path) if f.endswith('.npy')])
    pose_list = [np.load(f) @ cam_to_gripper_pose for f in pose_path_list]

    return {
        'fx': fx,
        'fy': fy,
        'ppx': ppx,
        'ppy': ppy,
        'depth_image_list': depth_image_list,
        'rgb_image_list': rgb_image_list,
        'pose_list': pose_list
    }

In [ ]:
dataset_dir = './data/objects_to_catch/'

dataset_dict = None

for seq_name in os.listdir(dataset_dir):
    seq_dir = os.path.join(dataset_dir, seq_name)
    if not os.path.isdir(seq_dir):
        continue
    print(f'Processing sequence {seq_name}')
    seq_data = gather_single_sequence_data(seq_dir)

    if dataset_dict is None:
        dataset_dict = seq_data
    else:
        assert np.isclose(dataset_dict['fx'], seq_data['fx'])
        assert np.isclose(dataset_dict['fy'], seq_data['fy'])
        assert np.isclose(dataset_dict['ppx'], seq_data['ppx'])
        assert np.isclose(dataset_dict['ppy'], seq_data['ppy'])
        assert dataset_dict['depth_image_list'][0].shape == seq_data['depth_image_list'][0].shape
        assert dataset_dict['rgb_image_list'][0].shape == seq_data['rgb_image_list'][0].shape
        dataset_dict['depth_image_list'] += seq_data['depth_image_list']
        dataset_dict['rgb_image_list'] += seq_data['rgb_image_list']
        dataset_dict['pose_list'] += seq_data['pose_list']

In [ ]:
volume = o3d.pipelines.integration.ScalableTSDFVolume(
    voxel_length=VOXEL_SIZE,
    sdf_trunc=3 * VOXEL_SIZE,
    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8
)

H, W = dataset_dict['depth_image_list'][0].shape

for idx in trange(len(dataset_dict['depth_image_list'])):
    pose = dataset_dict['pose_list'][idx]

    rgb = dataset_dict['rgb_image_list'][idx]
    rgb = np.ascontiguousarray(rgb)
    depth = dataset_dict['depth_image_list'][idx] / DEPTH_SCALING_FACTOR
    depth[depth > DEPTH_CUTOFF] = 0.0 # remove invalid depth
    depth = np.ascontiguousarray(depth.astype(np.float32))

    rgb = o3d.geometry.Image(rgb)
    depth = o3d.geometry.Image(depth)

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgb, depth, depth_scale=1.0, depth_trunc=4.0, convert_rgb_to_intensity=False)
    intrinsic = o3d.camera.PinholeCameraIntrinsic(width=W, height=H, fx=dataset_dict['fx'], fy=dataset_dict['fy'], cx=dataset_dict['ppx'], cy=dataset_dict['ppy'])
    extrinsic = np.linalg.inv(pose)
    # extrinsic = pose
    volume.integrate(rgbd, intrinsic, extrinsic)

In [ ]:
mesh = volume.extract_triangle_mesh()

In [ ]:
mesh.vertices

In [ ]:
# visualize mesh
o3d.visualization.draw_geometries([mesh])

In [ ]:
mesh_path = os.path.join('./extracted.ply')
o3d.io.write_triangle_mesh(mesh_path, mesh)